In [101]:
import pandas as pd
import numpy as np

In [102]:
import xml.etree.ElementTree as ET

In [107]:
tree = ET.parse("C:/Users/Sri/Google Drive/Data Science/Python/Data_formats/bio_xml_parse/original_data_jonas/abstracts.xml")
#tree = ET.parse("C:/Users/Sri/Google Drive/Data Science/Python/Data_formats/bio_xml_parse/original_data_jonas/full_papers.xml")

root = tree.getroot()

### Simpler tagging

In [108]:
sentences = []
negations = []
speculations = []
for sent in root.iter('sentence'):
    sentences.append({sent.attrib['id']:"".join(sent.itertext())})
    for cue in sent.findall("./xcope/cue/[@type='negation']..."):
        negations.append({sent.attrib['id']:"".join(cue.itertext())})
    for cue in sent.findall("./xcope/cue/[@type='speculation']..."):
        speculations.append({sent.attrib['id']:"".join(cue.itertext())})

In [109]:
sent_df = pd.DataFrame()
lol_sents = [list(elem.values()) for elem in sentences]
lol_keys = [list(elem.keys()) for elem in sentences]

sents = [y for x in lol_sents for y in x]
sid = [y for x in lol_keys for y in x]
sent_df['sid'] = sid
sent_df['sentences'] = sents

sent_df.head()

,sid,sentences
0,S1.1,Induction of NF-KB during monocyte differentia...
1,S1.2,The production of human immunodeficiency virus...
2,S1.3,Electrophoretic mobility shift assays and Sout...
3,S1.4,"PMA treatment, and not retinoic acid treatment..."
4,S1.5,In nuclear extracts from monocytes or macropha...


In [113]:
sent_df.shape

(11871, 2)

In [110]:
negations_df = pd.DataFrame()
lol_negscopes = [list(elem.values()) for elem in negations]
lol_negkeys = [list(elem.keys()) for elem in negations]

neg_scopes = [y for x in lol_negscopes for y in x]
neg_sid = [y for x in lol_negkeys for y in x]
negations_df['sid'] = neg_sid
negations_df['neg_scopes'] = neg_scopes
negations_df.head()

,sid,neg_scopes
0,S1.4,not retinoic acid treatment of the U937 cells
1,S1.6,no induction of NF-KB factor was detected
2,S2.3,not in Jurkat T cells or HeLa cells
3,S2.6,not in Jurkat or HeLa cells
4,S2.8,no enhancer activity of its own


In [111]:
speculations_df = pd.DataFrame()
lol_specscopes = [list(elem.values()) for elem in speculations]
lol_speckeys = [list(elem.keys()) for elem in speculations]

spec_scopes = [y for x in lol_specscopes for y in x]
spec_sid = [y for x in lol_speckeys for y in x]
speculations_df['sid'] = spec_sid
speculations_df['spec_scopes'] = spec_scopes
speculations_df.head()

,sid,spec_scopes
0,S1.6,suggesting that this factor was not required f...
1,S1.7,"indicate that in monocytic cell lineage, HIV-1..."
2,S2.4,"B cells, T cells, or HeLa cells"
3,S2.9,could suppress the basal-level activity of the...
4,S2.11,probably a target site for both positive and n...


In [112]:
bio_df = pd.merge(sent_df,negations_df, how='left')
bio_df_scopes = pd.merge(bio_df,speculations_df, how='left')
bio_df_scopes.shape

(12045, 4)

In [91]:
bio_df_scopes.head(3)

,sid,sentences,neg_scopes,spec_scopes
0,S1.1,Induction of NF-KB during monocyte differentia...,NaN,NaN
1,S1.2,The production of human immunodeficiency virus...,NaN,NaN
2,S1.3,Electrophoretic mobility shift assays and Sout...,NaN,NaN


In [92]:
bio_df_scopes.reset_index(drop=False, inplace=True)
bio_df_scopes.rename(columns={'index':'oidx'},inplace=True)
bio_df_scopes.head(5)

,oidx,sid,sentences,neg_scopes,spec_scopes
0,0,S1.1,Induction of NF-KB during monocyte differentia...,NaN,NaN
1,1,S1.2,The production of human immunodeficiency virus...,NaN,NaN
2,2,S1.3,Electrophoretic mobility shift assays and Sout...,NaN,NaN
3,3,S1.4,"PMA treatment, and not retinoic acid treatment...",not retinoic acid treatment of the U937 cells,NaN
4,4,S1.5,In nuclear extracts from monocytes or macropha...,NaN,NaN


In [93]:
bio_df_final = bio_df_scopes.groupby('sid').agg({ 'oidx':'first', 'sentences':'first','neg_scopes': list, 'spec_scopes':list})
bio_df_final.reset_index(inplace=True)
bio_df_final.sort_values('oidx', inplace=True)

In [94]:
bio_df_final.head(5)

,sid,oidx,sentences,neg_scopes,spec_scopes
0,S1.1,0,Induction of NF-KB during monocyte differentia...,[nan],[nan]
1,S1.2,1,The production of human immunodeficiency virus...,[nan],[nan]
2,S1.3,2,Electrophoretic mobility shift assays and Sout...,[nan],[nan]
3,S1.4,3,"PMA treatment, and not retinoic acid treatment...",[not retinoic acid treatment of the U937 cells],[nan]
4,S1.5,4,In nuclear extracts from monocytes or macropha...,[nan],[nan]


In [95]:
bio_df_final['neg_scopes'] = bio_df_final['neg_scopes'].apply(lambda x:[] if pd.isnull(x[0]) else x)
bio_df_final['spec_scopes'] = bio_df_final['spec_scopes'].apply(lambda x:[] if pd.isnull(x[0]) else x)

In [96]:
bio_df_final.iloc[100:150,:]

,sid,oidx,sentences,neg_scopes,spec_scopes
1015,S11.9,101,They bind to the kappa B motifs with different...,[],[may reflect their different contribution in t...
2012,S12.1,102,The functional domains of the murine Thy-1 gen...,[],[]
2013,S12.2,103,"The Thy-1 gene promoter resembles a "" housekee...",[lacks a canonical TATA box],[]
2014,S12.3,104,"Using transgenic mice, we show that this promo...",[not confer any tissue specificity],[]
2015,S12.4,105,It can only be activated in a tissue-specific ...,[],[]
2016,S12.5,106,We have analyzed the functional domains of the...,[],[]
2017,S12.6,107,DNase I and gel mobility shift assays show the...,[],[]
2018,S12.7,108,Our results show that the structure of this pr...,[],[]
2836,S13.1,109,Comparison of constitutive and inducible trans...,[],[]
2837,S13.2,110,The kappa B sequence (GGGACTTTCC) binds a fact...,[],[]


In [98]:
bio_df_final.shape

(11871, 5)

In [100]:
bio_df_final.to_excel('bio_df_final.xlsx', index=False)

In [ ]:
len(sentences)

In [ ]:
for scope in root.iter('cue'):
        print (scope.text)

In [ ]:
sentences = []
scopes = []

for sentence in root.iter('sentence'):
    for scope in sentence.iter():
        print (scope.text, scope.tag)
#         if(scope.tag == 'sentence'):
#             sentences.append(''.join(scope.itertext()))
#             scopes.append('')
#         elif(scope.tag == 'xcope'):
#             scopes.append(''.join(scope.itertext()))

In [ ]:
sentences

In [ ]:
scopes

In [ ]:
sentences

In [ ]:
scopes

In [ ]:
for movie in root.findall("./genre/decade/movie/format/[@multiple='Yes']..."):
    print(movie.attrib)

In [ ]:
import xml.etree.ElementTree as ET
#x =  # your xml file
tree = ET.parse("C:/Users/Sri/Google Drive/Data Science/Python/Data_formats/jonas_github/original_data_jonas/abstracts.xml")
root = tree.getroot()
string = ""
for c in root:
    string +=  c.text.strip()
print (string)

In [ ]:
print(ET.tostring(root, encoding='utf8').decode('utf8'))

In [ ]:
root.tag

In [ ]:
root.attrib

In [ ]:
for child in root:
    print (child.tag, child.attrib)

In [ ]:
#To see all the elements in the entire tree

In [ ]:
# helpfulway to see the whole document

In [ ]:
print(ET.tostring(root, encoding='utf8').decode('utf8'))

In [ ]:
def all_texts(root):
    for child in root:
         yield child.tail

In [ ]:
list(all_texts(doc))

In [ ]:
import xml.etree.ElementTree as et

xml = """
<A>
    hello

    <annotation> NOT part of text </annotation>

    world
</A>"""

In [ ]:
doc = et.fromstring(xml)

In [ ]:
def all_texts(root):
    if root.text is not None:
         yield root.text
    for child in root:
         if child.tail is not None:
            yield child.tail

In [ ]:
list(all_texts(doc))

In [ ]:
# To look inside a particular element

In [ ]:
len([sent.text for sent in root.iter('sentence')])

In [ ]:
for child in root.iter('sentence'):
    print (sent.tail.text)

In [ ]:
for child in root.findall("./DocumentSet/Document/DocID/DocumentPart/sentence"):
    print (child.attrib, child.text)

In [ ]:
scopes = [sent.text for sent in root.iter('xcope')]
scopes

In [ ]:
sents[3]

In [ ]:
sents[4]

In [ ]:
[movie.attrib for movie in root.iter('format')]

**XPath Expressions
Many times elements will not have attributes, they will only have text content. Using the attribute .text, you can print out this content.**

In [ ]:
for description in root.iter('description'):
    print(description.text)

In [ ]:
for format in root.iter('format'):
    print(format.text)

**Understanding XPath is critically important to scanning and populating XMLs. ElementTree has a .findall() function that will traverse the immediate children of the referenced element. You can use XPath expressions to specify more useful searches.**

In [ ]:
for movie in root.findall("./genre/decade/movie/[year='1992']"):
    print(movie.attrib)

**The function .findall() always begins at the element specified. This type of function is extremely powerful for a "find and replace". You can even search on attributes!
Now, print out only the movies that are available in multiple formats (an attribute).**

In [ ]:
for format in root.findall("./genre/decade/movie/format/[@multiple='Yes']"):
    print(format.attrib)

In [ ]:
# with findall you can traverse the children and also the attribues of an element

In [ ]:
for format in root.findall("./genre/decade/movie/[@multiple='Yes']"):
    print(format.attrib)
#we get nothing here because @multiple is not an attribue of movie nor its child.

In [ ]:
# to return the parent attribute of the current element use ...

In [ ]:
for movie in root.findall("./genre/decade/movie/format/[@multiple='Yes']..."):
    print(movie.attrib)

### Modifying an XML

In [ ]:
for movie in root.iter('movie'):
    print (movie.attrib)

In [ ]:
b2tf = root.find("./genre/decade/movie/[@title='Back 2 the Future']")
b2tf

Notice that using the .find() method returns an element of the tree. Much of the time, it is more useful to edit the content within an element.
Modify the title attribute of the Back 2 the Future element variable to read "Back to the Future". Then, print out the attributes of your variable to see your change. You can easily do this by accessing the attribute of an element and then assigning a new value to it:

In [ ]:
b2tf.attrib["title"] = "Back to the Future"
print(b2tf.attrib)

Write out your changes back to the XML so they are permanently fixed in the document. Print out your movie attributes again to make sure your changes worked. Use the .write() method to do this:

In [ ]:
tree.write("movies.xml") # this overwrites the xml file.
# interesting, we saved the change to a variable, but the original document
# got updated with the above command.

tree = ET.parse('movies.xml')
root = tree.getroot()

for movie in root.iter('movie'):
    print(movie.attrib)

Fixing Attributes
The multiple attribute is incorrect in some places. Use ElementTree to fix the designator based on how many formats the movie comes in. First, print the format attribute and text to see which parts need to be fixed.

In [ ]:
for form in root.findall("./genre/decade/movie/format"):
    print(form.attrib, form.text)

In [ ]:
import re

for form in root.findall("./genre/decade/movie/format"):
    # Search for the commas in the format text
    match = re.search(',',form.text)
    if match:
        form.set('multiple','Yes')
    else:
        form.set('multiple','No')

In [ ]:
# Write out the tree to the file again
tree.write("movies.xml")

tree = ET.parse('movies.xml')
root = tree.getroot()

for form in root.findall("./genre/decade/movie/format"):
    print(form.attrib, form.text)

Moving Elements
Some of the data has been placed in the wrong decade. Use what you have learned about XML and ElementTree to find and fix the decade data errors.

It will be useful to print out both the decade tags and the year tags throughout the document.

In [ ]:
for decade in root.findall("./genre/decade"):
    print(decade.attrib)
    for year in decade.findall("./movie/year"):
        print(year.text, '\n')

In [ ]:
for decade in root.iter('decade'):
    print(decade.attrib)

The two years that are in the wrong decade are the movies from the 2000s. Figure out what those movies are, using an XPath expression.

In [ ]:
for movie in root.findall("./genre/decade/movie/[year='2000']"):
    print (movie.attrib)

You have to add a new decade tag, the 2000s, to the Action genre in order to move the X-Men data. The .SubElement() method can be used to add this tag to the end of the XML.

In [ ]:
action = root.find("./genre[@category='Action']")
new_dec = ET.SubElement(action, 'decade')
new_dec.attrib["years"] = '2000s'

In [ ]:
print(ET.tostring(action, encoding='utf8').decode('utf8'))

Now append the X-Men movie to the 2000s and remove it from the 1990s, using .append() and .remove(), respectively.

In [ ]:
xmen = root.find("./genre/decade/movie[@title='X-Men']")
dec2000s = root.find("./genre[@category='Action']/decade[@years='2000s']")
dec2000s.append(xmen)
dec1990s = root.find("./genre[@category='Action']/decade[@years='1990s']")
dec1990s.remove(xmen)

In [ ]:
print(ET.tostring(action, encoding='utf8').decode('utf8'))

Build XML Documents
Nice, so you were able to essentially move an entire movie to a new decade. Save your changes back to the XML.

In [ ]:
tree.write("movies.xml")

tree = ET.parse('movies.xml')
root = tree.getroot()

print(ET.tostring(root, encoding='utf8').decode('utf8'))